# Few fourier = simple coil geometry?
Set up the simple torus test cases from the efficient fields paper. We will assume no additional Field from the Plasma (No $B_{external}$ as in the advanced example) so our coils simply have to produce $B.n=0$ on our **computational boundary(?)** which we say is a constant offset away from the plasma boundary

![efficient_fields_paper_test_cases](efficient_fields_paper_test_cases.png)

The following code is mostly copied from the `stage_two_optimization.py` example:

In [ ]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from scipy.optimize import minimize
from simsopt.field import BiotSavart, Current, coils_via_symmetries
from simsopt.geo import (SurfaceRZFourier, curves_to_vtk, create_equally_spaced_curves,
                         CurveLength, CurveCurveDistance, MeanSquaredCurvature,
                         LpCurveCurvature, CurveSurfaceDistance)
from simsopt.objectives import Weight, SquaredFlux, QuadraticPenalty

# Number of unique coil shapes, i.e. the number of coils per half field period:
# (Since the configuration has nfp = 2, multiply by 4 to get the total number of coils.)
ncoils = 3

# Major radius for the initial circular coils:
R0 = 1.0

# Minor radius for the initial circular coils:
R1 = 0.5

# Number of Fourier modes describing each Cartesian component of each coil:
order = 5

# Threshold and weight for the coil-to-coil distance penalty in the objective function:
CC_THRESHOLD = 0.1
CC_WEIGHT = 1000

# Threshold and weight for the coil-to-surface distance penalty in the objective function:
CS_THRESHOLD = 0.3
CS_WEIGHT = 10

# Threshold and weight for the curvature penalty in the objective function:
CURVATURE_THRESHOLD = 5.
CURVATURE_WEIGHT = 1e-6

# Threshold and weight for the mean squared curvature penalty in the objective function:
MSC_THRESHOLD = 5
MSC_WEIGHT = 1e-6


# Number of iterations to perform:
MAXITER = 50

# Directory for output
OUT_DIR = "./output/"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
def optimize(s:SurfaceRZFourier):
    LENGTH_WEIGHT = Weight(1e-6)
    # Weight on the curve lengths in the objective function. We use the `Weight`
    # class here to later easily adjust the scalar value and rerun the optimization
    # without having to rebuild the objective.
    # Initialize the boundary magnetic surface:

    # Create the initial coils:
    base_curves = create_equally_spaced_curves(ncoils, s.nfp, stellsym=True, R0=R0, R1=R1, order=order)
    base_currents = [Current(1e5) for i in range(ncoils)]
    # Since the target field is zero, one possible solution is just to set all
    # currents to 0. To avoid the minimizer finding that solution, we fix one
    # of the currents:
    base_currents[0].fix_all()

    coils = coils_via_symmetries(base_curves, base_currents, s.nfp, True)
    bs = BiotSavart(coils)
    bs.set_points(s.gamma().reshape((-1, 3)))

    curves = [c.curve for c in coils]
    
    # Define the individual terms objective function:
    Jf = SquaredFlux(s, bs)
    Jls = [CurveLength(c) for c in base_curves]
    Jccdist = CurveCurveDistance(curves, CC_THRESHOLD, num_basecurves=ncoils)
    Jcsdist = CurveSurfaceDistance(curves, s, CS_THRESHOLD)
    Jcs = [LpCurveCurvature(c, 2, CURVATURE_THRESHOLD) for c in base_curves]
    Jmscs = [MeanSquaredCurvature(c) for c in base_curves]


    # Form the total objective function. To do this, we can exploit the
    # fact that Optimizable objects with J() and dJ() functions can be
    # multiplied by scalars and added:
    JF = Jf \
        + LENGTH_WEIGHT * sum(Jls) \
        + CC_WEIGHT * Jccdist \
        + CS_WEIGHT * Jcsdist \
        + CURVATURE_WEIGHT * sum(Jcs) \
        + MSC_WEIGHT * sum(QuadraticPenalty(J, MSC_THRESHOLD, "max") for J in Jmscs)

    # We don't have a general interface in SIMSOPT for optimisation problems that
    # are not in least-squares form, so we write a little wrapper function that we
    # pass directly to scipy.optimize.minimize


    def fun(dofs):
        JF.x = dofs
        J = JF.J()
        grad = JF.dJ()
        jf = Jf.J()
        BdotN = np.mean(np.abs(np.sum(bs.B().reshape(s.unitnormal().shape) * s.unitnormal(), axis=2)))
        return J, grad

    dofs = JF.x
    
    print("### Run the optimisation #####################################################")
    res = minimize(fun, dofs, jac=True, method='L-BFGS-B', options={'maxiter': MAXITER, 'maxcor': 300}, tol=1e-15)


    # We now use the result from the optimization as the initial guess for a
    # subsequent optimization with reduced penalty for the coil length. This will
    # result in slightly longer coils but smaller `B·n` on the surface.
    dofs = res.x
    LENGTH_WEIGHT *= 0.1
    res = minimize(fun, dofs, jac=True, method='L-BFGS-B', options={'maxiter': MAXITER, 'maxcor': 300}, tol=1e-15)

    bs.save(OUT_DIR + "biot_savart_opt.json")


    # Define the individual terms objective function:
    Jf = SquaredFlux(s, bs)
    Jls = [CurveLength(c) for c in base_curves]
    Jccdist = CurveCurveDistance(curves, CC_THRESHOLD, num_basecurves=ncoils)
    Jcsdist = CurveSurfaceDistance(curves, s, CS_THRESHOLD)
    Jcs = [LpCurveCurvature(c, 2, CURVATURE_THRESHOLD) for c in base_curves]
    Jmscs = [MeanSquaredCurvature(c) for c in base_curves]

    return coils, float(JF.J()), float(Jf.J())


Iteratively increase the m and n resolution, and watch the complexity of the geometry increase

R perturbations:

In [ ]:
%matplotlib inline
import simsopt.geo.plotting as spl

Mpol_max = 6 # theta
Ntor_max = 5 # phi
nfp = 3

torus_surface = SurfaceRZFourier.from_nphi_ntheta(64, 64, "field period", nfp)
mpol_start = torus_surface.mpol
ntor_start = torus_surface.ntor

all_coils = []
for m in range(mpol_start, Mpol_max):
  for n in range(ntor_start, Ntor_max):
    for sig_n in [1, -1]:
      torus_surface = SurfaceRZFourier.from_nphi_ntheta(64, 64, "field period", nfp)
      torus_surface.change_resolution(m,n)
      for perturbation in [1e-2]: #[-1e-2,-1e-3,1e-3,1e-2] # Perturbation in +- direction seems to give same result
        if(torus_surface.get_rc(m,n) != 0):
          continue # Skip the required modes to get any torus at all
        torus_surface.set_rc(m,sig_n*n, perturbation)

        # if(torus_surface.get_zs(m,n) != 0):
        #   continue # Skip the required modes to get any torus at all
        # torus_surface.set_zs(m,sig_n*n, perturbation)

        coils, score, Bn_residual = optimize(torus_surface)
        all_coils.append({"coils":coils,
                          "surf":torus_surface,
                          "m":m,"n":sig_n*n,
                          "perturbation":perturbation,
                          "final_score":score,
                          "Bn":Bn_residual})
        spl.fix_matplotlib_3d(spl.plot([torus_surface]+coils))
    # torus_surface.plot()


In [ ]:
all_coils_z = []
for m in range(mpol_start, Mpol_max):
  for n in range(ntor_start, Ntor_max):
    for sig_n in [1, -1]:
      torus_surface = SurfaceRZFourier.from_nphi_ntheta(64, 64, "field period", nfp)
      torus_surface.change_resolution(m,n)
      for perturbation in [1e-2]: #[-1e-2,-1e-3,1e-3,1e-2] # Perturbation in +- direction seems to give same result
        # if(torus_surface.get_rc(m,n) != 0):
        #   continue # Skip the required modes to get any torus at all
        # torus_surface.set_rc(m,sig_n*n, perturbation)

        if(torus_surface.get_zs(m,n) != 0):
          continue # Skip the required modes to get any torus at all
        torus_surface.set_zs(m,sig_n*n, perturbation)

        coils, score, Bn_residual = optimize(torus_surface)
        all_coils_z.append({"coils":coils,
                          "surf":torus_surface,
                          "m":m,"n":sig_n*n,
                          "perturbation":perturbation,
                          "final_score":score,
                          "Bn":Bn_residual})
        spl.fix_matplotlib_3d(spl.plot([torus_surface]+coils))
    # torus_surface.plot()


In [ ]:
import pandas as pd
df = pd.DataFrame(all_coils)
df["coil_complexity"] = df["final_score"] - df["Bn"] 
df_z = pd.DataFrame(all_coils_z)
df_z["coil_complexity"] = df_z["final_score"] - df_z["Bn"] 

def scoring_func(list_of_coils, surf):
  print(list_of_coils, surf)
  curves = [c.curve for c in list_of_coils]
  Jccdist = CurveCurveDistance(curves, CC_THRESHOLD, num_basecurves=ncoils)
  # Jcsdist = CurveSurfaceDistance(curves, s, CS_THRESHOLD)
  Jcs = [LpCurveCurvature(c, 2, CURVATURE_THRESHOLD) for c in curves]
  Jmscs = [MeanSquaredCurvature(c) for c in curves]

  # Form the total objective function. To do this, we can exploit the
  # fact that Optimizable objects with J() and dJ() functions can be
  # multiplied by scalars and added:
  JF_eval = CC_WEIGHT * Jccdist \
      + CURVATURE_WEIGHT * sum(Jcs) \
      + MSC_WEIGHT * sum(QuadraticPenalty(J, MSC_THRESHOLD, "max") for J in Jmscs)
  return JF_eval.J()

#df["score"] = df["coils"].apply(scoring_func)

df.head()

In [ ]:
row = df[(df["m"]==5) & (df["n"] == 0)]

spl.fix_matplotlib_3d(spl.plot([row["surf"].iat[0]]+row["coils"].iat[0]))

In [ ]:
row

In [ ]:
import plotly.express as px
px.scatter_3d(df, "m", "n", "Bn", "coil_complexity").show()
px.scatter_3d(df_z, "m", "n", "Bn", "coil_complexity")

In [ ]:
%matplotlib widget
plt.close("all")
spl.fix_matplotlib_3d(spl.plot([torus_surface]+coils))